# Apurva Ukande

# MoEngage - PushAmp+ Case Study

## Importing required libraries in Notebook

In [1]:
# Importing Libraries
import numpy as np   
import pandas as pd    
import matplotlib.pyplot as plt   
import seaborn as sns

In [2]:
import json
import re

## Loading json to Pandas Dataframe

In [3]:
# Importing json file and converting it into Dataframe
d = open('PA_test_data.json')
data = json.load(d)
df = pd.DataFrame(data)


In [4]:
# Checking top 5 records
df.head()

,ctype,cid,delivery_type,sherpa_flag,target_segment,ttl,date,sent,impressions,paplus_sent,imp,app_name,chk_flag
1,TXN API Push,5f76dca5d69d380756710da6,later,SMV,custom_segment,43200.0,2020-11-11,0.0,6.0,0.0,0.0,Songswala,6.0
2,TXN API Push,5d95c4c4287853101776a97a,soon,SMV,User,43200.0,2020-11-15,77993.0,70934.0,0.0,0.0,Sabziwala,148927.0
3,ST Push,5e8b11e4e3ca782085182ce1,later,0,custom_filters,21600.0,2020-11-12,3344.0,2457.0,0.0,0.0,TVwala,5801.0
6,TXN API Push,5fc17fda119bd90684d75d03,soon,SMV,User,43200.0,2020-11-27,2.0,1.0,0.0,0.0,Armwala,3.0
7,TXN API Push,5ed0e1dba161e50754b65400,soon,SMV,User,129600.0,2020-11-07,0.0,1.0,0.0,0.0,TVwala,1.0


In [5]:
# Checking bottom 5 records
df.tail()

,ctype,cid,delivery_type,sherpa_flag,target_segment,ttl,date,sent,impressions,paplus_sent,imp,app_name,chk_flag
166942,ST Push,5f59016fb7f37c13ecaeb066,later,DMV,allusers,129600.0,2020-11-10,33001.0,15124.0,0.0,0.0,Phonewala,48125.0
166943,TXN API Push,5f89d75aec3c9a074e59edf4,later,SMV,custom_segment,14400.0,2020-11-30,0.0,14.0,0.0,0.0,Songswala,14.0
166945,ST Push,5de64bee20fdc014fb87bbf0,later,0,allusers,129600.0,2020-11-26,0.0,4.0,0.0,0.0,Phonewala,4.0
166947,ST Push,5e8b33216b0c154e2be105e8,later,0,allusers,129600.0,2020-11-04,0.0,40.0,0.0,0.0,Phonewala,40.0
166948,General Push,5fbf59f4f8861b5da664fb37,later,0,custom_filters,129600.0,2020-11-26,150895.0,92085.0,0.0,0.0,Life_max,242980.0


#### Observation: Data has been loaded correctly into the pandas dataframe

## Retreiving the list of fields along with their data type

In [6]:
# Size of the dataset
df.shape

(106699, 13)

In [7]:
# Concise summary of a DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 106699 entries, 1 to 166948
Data columns (total 13 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   ctype           106699 non-null  object 
 1   cid             106699 non-null  object 
 2   delivery_type   106699 non-null  object 
 3   sherpa_flag     106699 non-null  object 
 4   target_segment  106699 non-null  object 
 5   ttl             106699 non-null  float64
 6   date            106699 non-null  object 
 7   sent            106699 non-null  float64
 8   impressions     106699 non-null  float64
 9   paplus_sent     106699 non-null  float64
 10  imp             106699 non-null  float64
 11  app_name        106699 non-null  object 
 12  chk_flag        106699 non-null  float64
dtypes: float64(6), object(7)
memory usage: 11.4+ MB


#### Observation:
    1. Column type are in expected format.
    2. There are no missing values, as Non-Null count is equal to number of entries( row ).

In [8]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
ttl,106699.0,105578.603923,5.065333e+04,0.0,43200.0,129600.0,129600.0,1800000.0
sent,106699.0,115843.132775,1.311482e+06,0.0,0.0,0.0,1186.5,74959486.0
impressions,106699.0,52844.579734,5.400473e+05,0.0,1.0,7.0,663.0,26038486.0
paplus_sent,106699.0,4910.934095,9.055159e+04,0.0,0.0,0.0,0.0,4658017.0
imp,106699.0,3628.216637,6.527142e+04,0.0,0.0,0.0,0.0,3339745.0
chk_flag,106699.0,168687.712509,1.829556e+06,1.0,2.0,9.0,1987.0,96826791.0


#### Observation
        1. Min of all numeric column is not negative, as expected.
        2. Mean of sent is greater than mean of impressions.

## Data Pre-processing

## Eliminating test DBs, the app name ending with 'moetest', from dataset

In [9]:
# l is a list containg true for rows that don't contain 'moetest'
# and containing False for the rows containing 'moetest'
l = []
for i in df['app_name']:
    if re.search('.+moetest', i.lower()):
        l.append(False)
    else:
        l.append(True)
        
        
def unique(l):
    x = np.array(l)
    print(np.unique(x))
    
# checking if any row actually contains 'moetest' or not    
unique(l)

[False  True]


In [10]:
# Eliminating 'moetest' rows from main dataset (df)
df = df[l]

In [11]:
#Verifying all unique app name for absense of 'moetest'
df.app_name.unique()

array(['Songswala', 'Sabziwala', 'TVwala', 'Armwala', 'Gharwala',
       'Phonewala', 'Clothwala', 'Gharwala_2', 'ABC_Prod', 'Set_IND',
       'Wholesale_prod', 'babywala', 'Dailyprod', 'Malay_prod',
       'UAE_Prod', 'Life_IND', 'Daily2_production', 'Life_max',
       'TripTest', 'Book_test', 'Style_IND', 'Scooterportal',
       'Fashion4prod', 'MMS', 'paintIND', 'ShoesKUL', 'RVAPROD',
       'LeranNow', 'BitcoinCX', 'GoldProduction', 'ChampionProd',
       'id_dealers', 'nineten_ind', 'astro_hindi', 'apmcprod', 'box_prod',
       'scootersPROD', 'uae_cluster_uat', 'open_mindapp', 'bikebuddy',
       'biggerprod', 'dokterkonsult'], dtype=object)

In [12]:
df.head()

,ctype,cid,delivery_type,sherpa_flag,target_segment,ttl,date,sent,impressions,paplus_sent,imp,app_name,chk_flag
1,TXN API Push,5f76dca5d69d380756710da6,later,SMV,custom_segment,43200.0,2020-11-11,0.0,6.0,0.0,0.0,Songswala,6.0
2,TXN API Push,5d95c4c4287853101776a97a,soon,SMV,User,43200.0,2020-11-15,77993.0,70934.0,0.0,0.0,Sabziwala,148927.0
3,ST Push,5e8b11e4e3ca782085182ce1,later,0,custom_filters,21600.0,2020-11-12,3344.0,2457.0,0.0,0.0,TVwala,5801.0
6,TXN API Push,5fc17fda119bd90684d75d03,soon,SMV,User,43200.0,2020-11-27,2.0,1.0,0.0,0.0,Armwala,3.0
7,TXN API Push,5ed0e1dba161e50754b65400,soon,SMV,User,129600.0,2020-11-07,0.0,1.0,0.0,0.0,TVwala,1.0


## Treating Bad Data

#### For every row of dataset : sent number should be greater than or equal to impressions.

In [13]:
# Examine dataset to find out rows where number of sent is less than impressions.
temp = df[df['sent'] < df['impressions']]
temp

,ctype,cid,delivery_type,sherpa_flag,target_segment,ttl,date,sent,impressions,paplus_sent,imp,app_name,chk_flag
1,TXN API Push,5f76dca5d69d380756710da6,later,SMV,custom_segment,43200.0,2020-11-11,0.0,6.0,0.0,0.0,Songswala,6.0
7,TXN API Push,5ed0e1dba161e50754b65400,soon,SMV,User,129600.0,2020-11-07,0.0,1.0,0.0,0.0,TVwala,1.0
9,TXN API Push,5f7ad696d7dfaa075bea0ada,soon,SMV,User,129600.0,2020-11-05,0.0,3.0,0.0,0.0,TVwala,3.0
14,TXN API Push,5f7c8f44ec29040756801736,later,SMV,custom_segment,43200.0,2020-11-30,0.0,13.0,0.0,0.0,Songswala,13.0
15,TXN API Push,5f97d7984b80c908ea9772ef,soon,SMV,User,129600.0,2020-11-28,0.0,6.0,0.0,0.0,TVwala,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
166938,ST Push,5eaffe08f6081560e8416ea8,later,DMV,allusers,129600.0,2020-11-15,0.0,3.0,0.0,0.0,Phonewala,3.0
166939,ST Push,5ec29279ee236e3a5f2d6a7e,later,DMV,allusers,129600.0,2020-11-28,0.0,3.0,0.0,0.0,Phonewala,3.0
166943,TXN API Push,5f89d75aec3c9a074e59edf4,later,SMV,custom_segment,14400.0,2020-11-30,0.0,14.0,0.0,0.0,Songswala,14.0
166945,ST Push,5de64bee20fdc014fb87bbf0,later,0,allusers,129600.0,2020-11-26,0.0,4.0,0.0,0.0,Phonewala,4.0


#### Observation : There are 62416 entries in the dataset where sent number is less than impressions, indicating bad data.

#### Dropping all rows from dataset where sent number is less than impressions

In [14]:
# Replacing dataset by eliminating bad data
df = df[df['sent'] >= df['impressions']]
df

,ctype,cid,delivery_type,sherpa_flag,target_segment,ttl,date,sent,impressions,paplus_sent,imp,app_name,chk_flag
2,TXN API Push,5d95c4c4287853101776a97a,soon,SMV,User,43200.0,2020-11-15,77993.0,70934.0,0.0,0.0,Sabziwala,148927.0
3,ST Push,5e8b11e4e3ca782085182ce1,later,0,custom_filters,21600.0,2020-11-12,3344.0,2457.0,0.0,0.0,TVwala,5801.0
6,TXN API Push,5fc17fda119bd90684d75d03,soon,SMV,User,43200.0,2020-11-27,2.0,1.0,0.0,0.0,Armwala,3.0
13,General Push,5fb11cd3bc2f79154e7ca549,later,0,custom_filters,129600.0,2020-11-16,14759.0,10643.0,0.0,0.0,Gharwala,25402.0
17,ST Push,5f8eda9c97223f714dce0ad8,later,0,allusers,129600.0,2020-11-26,1092.0,537.0,0.0,0.0,Phonewala,1629.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
166936,ST Push,5f51dc9e4371644051943e19,later,0,allusers,129600.0,2020-11-25,292916.0,115285.0,0.0,0.0,Phonewala,408201.0
166937,ST Push,5f59016fb7f37c13ecaeb066,later,DMV,allusers,129600.0,2020-11-19,49829.0,21796.0,0.0,0.0,Phonewala,71625.0
166941,General Push,5fa3ab20a6e83c4fc286faa3,later,0,custom_filters,86400.0,2020-11-05,4612.0,3667.0,0.0,0.0,Life_max,8279.0
166942,ST Push,5f59016fb7f37c13ecaeb066,later,DMV,allusers,129600.0,2020-11-10,33001.0,15124.0,0.0,0.0,Phonewala,48125.0


#### For every row of dataset : sent number using PushAmp+ should be greater than or equal to impressions of PumpAmp+

In [15]:
# Examine dataset to find out rows where number of sent using PushAmp+ is less than impressions of PushAmp+
temp = df[df['paplus_sent'] < df['imp']]
temp

,ctype,cid,delivery_type,sherpa_flag,target_segment,ttl,date,sent,impressions,paplus_sent,imp,app_name,chk_flag
45461,General Push,5fbfa0b2f8861b2f3d3eaffc,soon,0,custom_filters,14400.0,2020-11-26,16882.0,9334.0,1783.0,1901.0,Songswala,26216.0
87888,General Push,5fbcaf7a5640b13e25e54d09,soon,0,custom_filters,14400.0,2020-11-24,16494.0,8806.0,1562.0,1615.0,Songswala,25300.0
135859,General Push,5fbf3d38df0f1720567c5af1,later,0,custom_filters,54000.0,2020-11-27,266292.0,231749.0,59536.0,60399.0,Malay_prod,498041.0
154441,General Push,5fbce9d35640b11605d64e25,soon,0,custom_filters,21600.0,2020-11-24,40350.0,7179.0,2367.0,2441.0,Songswala,47529.0


#### Observation : We observe some rows are not as per expectation
        

In [16]:
# Dropping bad rows
df = df[df['paplus_sent'] >= df['imp']]
df

,ctype,cid,delivery_type,sherpa_flag,target_segment,ttl,date,sent,impressions,paplus_sent,imp,app_name,chk_flag
2,TXN API Push,5d95c4c4287853101776a97a,soon,SMV,User,43200.0,2020-11-15,77993.0,70934.0,0.0,0.0,Sabziwala,148927.0
3,ST Push,5e8b11e4e3ca782085182ce1,later,0,custom_filters,21600.0,2020-11-12,3344.0,2457.0,0.0,0.0,TVwala,5801.0
6,TXN API Push,5fc17fda119bd90684d75d03,soon,SMV,User,43200.0,2020-11-27,2.0,1.0,0.0,0.0,Armwala,3.0
13,General Push,5fb11cd3bc2f79154e7ca549,later,0,custom_filters,129600.0,2020-11-16,14759.0,10643.0,0.0,0.0,Gharwala,25402.0
17,ST Push,5f8eda9c97223f714dce0ad8,later,0,allusers,129600.0,2020-11-26,1092.0,537.0,0.0,0.0,Phonewala,1629.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
166936,ST Push,5f51dc9e4371644051943e19,later,0,allusers,129600.0,2020-11-25,292916.0,115285.0,0.0,0.0,Phonewala,408201.0
166937,ST Push,5f59016fb7f37c13ecaeb066,later,DMV,allusers,129600.0,2020-11-19,49829.0,21796.0,0.0,0.0,Phonewala,71625.0
166941,General Push,5fa3ab20a6e83c4fc286faa3,later,0,custom_filters,86400.0,2020-11-05,4612.0,3667.0,0.0,0.0,Life_max,8279.0
166942,ST Push,5f59016fb7f37c13ecaeb066,later,DMV,allusers,129600.0,2020-11-10,33001.0,15124.0,0.0,0.0,Phonewala,48125.0


#### Total sent should be greater than sent by using PushAmp+

In [17]:
# Checking for violation of above rule
temp = df[df['sent'] < df['paplus_sent']]
temp

,ctype,cid,delivery_type,sherpa_flag,target_segment,ttl,date,sent,impressions,paplus_sent,imp,app_name,chk_flag


#### Observation : No bad data observed

## Verifying expected entries of all columns with data set.

In [18]:
df.ctype.unique()

array(['TXN API Push', 'ST Push', 'General Push'], dtype=object)

In [19]:
df.delivery_type.unique()

array(['soon', 'later', 'periodic'], dtype=object)

In [20]:
df.sherpa_flag.unique()

array(['SMV', 0, 'DMV'], dtype=object)

In [21]:
df.target_segment.unique()

array(['User', 'custom_filters', 'allusers', 'custom_segment'],
      dtype=object)

In [22]:
df.ttl.unique()

array([4.320e+04, 2.160e+04, 1.296e+05, 1.728e+05, 1.800e+04, 8.640e+04,
       1.440e+04, 1.080e+04, 5.400e+04, 2.880e+04, 7.200e+03, 6.120e+04,
       0.000e+00, 5.040e+04, 1.800e+06, 3.564e+05, 3.600e+03, 3.960e+04,
       1.800e+03, 2.592e+05, 2.400e+03, 3.600e+04, 3.240e+04, 5.400e+03,
       2.520e+04, 1.080e+05, 6.000e+01, 1.260e+05, 6.300e+03, 9.000e+03,
       6.480e+04, 6.840e+04])

In [23]:
df.date.unique()

array(['2020-11-15', '2020-11-12', '2020-11-27', '2020-11-16',
       '2020-11-26', '2020-11-28', '2020-11-29', '2020-11-24',
       '2020-11-13', '2020-11-05', '2020-11-08', '2020-11-18',
       '2020-11-17', '2020-11-07', '2020-11-23', '2020-11-14',
       '2020-11-09', '2020-11-10', '2020-11-01', '2020-11-06',
       '2020-11-03', '2020-11-21', '2020-11-11', '2020-11-20',
       '2020-11-19', '2020-11-30', '2020-11-02', '2020-11-04',
       '2020-11-25', '2020-11-22'], dtype=object)

#### Columns data entries of ctype, delivery type, sherpa flag, target segment, ttl and date are as expected.

#### Checking uniqueness of 'cid'

In [24]:
len(df.cid.unique())

23329

In [25]:
df.shape

(43854, 13)

#### The number of unique 'cid' is less than number of rows, this means some rows have same 'cid'. These are the due to the repetitive campaigns

In [26]:
df.sort_values('cid')

,ctype,cid,delivery_type,sherpa_flag,target_segment,ttl,date,sent,impressions,paplus_sent,imp,app_name,chk_flag
25891,ST Push,567bb39dc211fc6da05e0bf6,soon,SMV,custom_filters,0.0,2020-11-05,1.0,0.0,0.0,0.0,bikebuddy,1.0
15452,ST Push,569f7594c211fc43cf5bfe2f,soon,SMV,custom_filters,129600.0,2020-11-09,209682.0,57317.0,0.0,0.0,Songswala,266999.0
83737,ST Push,569f7594c211fc43cf5bfe2f,soon,SMV,custom_filters,129600.0,2020-11-29,115430.0,61339.0,0.0,0.0,Songswala,176769.0
81123,ST Push,569f7594c211fc43cf5bfe2f,soon,SMV,custom_filters,129600.0,2020-11-01,164873.0,47731.0,0.0,0.0,Songswala,212604.0
97468,ST Push,569f7594c211fc43cf5bfe2f,soon,SMV,custom_filters,129600.0,2020-11-22,112797.0,50495.0,0.0,0.0,Songswala,163292.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
164777,General Push,5fc56bff53bf566b8737c346,later,0,custom_filters,129600.0,2020-11-30,34705.0,18590.0,0.0,0.0,Clothwala,53295.0
128515,General Push,5fc56c4daa3ed66ab36d5146,later,0,custom_filters,129600.0,2020-11-30,13219.0,10072.0,0.0,0.0,Clothwala,23291.0
40394,TXN API Push,5fc57277f5b5a206703d6a11,soon,SMV,User,43200.0,2020-11-30,1.0,1.0,0.0,0.0,Armwala,2.0
82488,TXN API Push,5fc582e160b0470670417726,soon,SMV,User,43200.0,2020-11-30,1.0,1.0,0.0,0.0,Armwala,2.0


#### It's given that 'General Push' cannot be 'periodic'. We verify the same

In [27]:
temp = df[df['ctype'] == 'General Push']
temp = df[df['delivery_type'] == 'periodic']
temp

,ctype,cid,delivery_type,sherpa_flag,target_segment,ttl,date,sent,impressions,paplus_sent,imp,app_name,chk_flag
83,General Push,5fbb9515f3b6020b230c2664,periodic,0,custom_filters,18000.0,2020-11-23,40804.0,29812.0,0.0,0.0,Songswala,70616.0
105,General Push,5fa7f2fcf3b6023ed4080129,periodic,DMV,custom_filters,129600.0,2020-11-08,127290.0,40878.0,0.0,0.0,Phonewala,168168.0
216,General Push,5fae479df3b6024e6928a63f,periodic,DMV,custom_filters,129600.0,2020-11-13,120240.0,55122.0,0.0,0.0,Phonewala,175362.0
259,General Push,5fc35c18f3b602297ba6a24e,periodic,DMV,custom_filters,129600.0,2020-11-29,151775.0,79075.0,0.0,0.0,Phonewala,230850.0
260,General Push,5fa0f8f8f3b6024502bb91c7,periodic,DMV,custom_filters,129600.0,2020-11-03,6365.0,3233.0,0.0,0.0,Phonewala,9598.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
166821,General Push,5fb50a71f3b6024e6928d610,periodic,0,custom_filters,14400.0,2020-11-18,4342.0,1183.0,0.0,0.0,Songswala,5525.0
166845,General Push,5fbf6e9df3b602297ba687e5,periodic,0,custom_filters,129600.0,2020-11-26,8785.0,6991.0,0.0,0.0,GoldProduction,15776.0
166856,General Push,5facf99df3b6024e69289da7,periodic,0,custom_filters,129600.0,2020-11-12,540.0,440.0,0.0,0.0,GoldProduction,980.0
166886,General Push,5fc1ee76f3b602297ba698fb,periodic,DMV,custom_filters,129600.0,2020-11-28,731641.0,329375.0,0.0,0.0,Phonewala,1061016.0


#### We observe some bad data and need to drop it

In [28]:
# construct an list like object dictating which rows to keep
temp =  ~( (df['ctype'] == 'General Push') & (df['delivery_type'] == 'periodic' ) )
df = df[temp]
df

,ctype,cid,delivery_type,sherpa_flag,target_segment,ttl,date,sent,impressions,paplus_sent,imp,app_name,chk_flag
2,TXN API Push,5d95c4c4287853101776a97a,soon,SMV,User,43200.0,2020-11-15,77993.0,70934.0,0.0,0.0,Sabziwala,148927.0
3,ST Push,5e8b11e4e3ca782085182ce1,later,0,custom_filters,21600.0,2020-11-12,3344.0,2457.0,0.0,0.0,TVwala,5801.0
6,TXN API Push,5fc17fda119bd90684d75d03,soon,SMV,User,43200.0,2020-11-27,2.0,1.0,0.0,0.0,Armwala,3.0
13,General Push,5fb11cd3bc2f79154e7ca549,later,0,custom_filters,129600.0,2020-11-16,14759.0,10643.0,0.0,0.0,Gharwala,25402.0
17,ST Push,5f8eda9c97223f714dce0ad8,later,0,allusers,129600.0,2020-11-26,1092.0,537.0,0.0,0.0,Phonewala,1629.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
166936,ST Push,5f51dc9e4371644051943e19,later,0,allusers,129600.0,2020-11-25,292916.0,115285.0,0.0,0.0,Phonewala,408201.0
166937,ST Push,5f59016fb7f37c13ecaeb066,later,DMV,allusers,129600.0,2020-11-19,49829.0,21796.0,0.0,0.0,Phonewala,71625.0
166941,General Push,5fa3ab20a6e83c4fc286faa3,later,0,custom_filters,86400.0,2020-11-05,4612.0,3667.0,0.0,0.0,Life_max,8279.0
166942,ST Push,5f59016fb7f37c13ecaeb066,later,DMV,allusers,129600.0,2020-11-10,33001.0,15124.0,0.0,0.0,Phonewala,48125.0


In [29]:
# Check if any record shows someone who didn't sent any notifications at all
temp = df[df['sent'] == 0.0]
temp

,ctype,cid,delivery_type,sherpa_flag,target_segment,ttl,date,sent,impressions,paplus_sent,imp,app_name,chk_flag


In [30]:
#df.to_csv('MoEngage.csv')

#### Everyone is using the system at least once

## Analysis to quantify performance of PushAmp+

In [31]:
# We create some metric to observe the performance

# TS     : Total Sent
# Psent  : Notifications sent with PushAmp+
# NPSent : Notifications not sent with PushAmp+

# TImp   : Total Impression
# PImp   : Impressions due to PushAmp+
# NPImp  : Impresssions not due to PushAmp+

df['NPImp/NPSent'] = ((df['impressions']-df['imp'])/(df['sent'] - df['paplus_sent'])).round(2) #1
df['PSent/TS'] = ((df['paplus_sent']) / (df['sent']) ).round(2)                                #2
df['PImp/PSent'] =( (df['imp']) / (df['paplus_sent']+1) ).round(2)                             #3

df['TImp/TS'] = ((df['impressions']) / (df['sent']) ).round(2)                                 #4

# In a similar scenario for a given app/situation, we observe (#1)% of non-PushAmp+ delivery rate
# But if we add (#2)% of messages using PushAmp+, those PushAmp+ messages themselves
# observe (#3)% delivery rate, benifitting overall deliver rate, ie (#4)

df

<ipython-input-31-df3210276887>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['NPImp/NPSent'] = ((df['impressions']-df['imp'])/(df['sent'] - df['paplus_sent'])).round(2) #1
<ipython-input-31-df3210276887>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['PSent/TS'] = ((df['paplus_sent']) / (df['sent']) ).round(2)                                #2
<ipython-input-31-df3210276887>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

,ctype,cid,delivery_type,sherpa_flag,target_segment,ttl,date,sent,impressions,paplus_sent,imp,app_name,chk_flag,NPImp/NPSent,PSent/TS,PImp/PSent,TImp/TS
2,TXN API Push,5d95c4c4287853101776a97a,soon,SMV,User,43200.0,2020-11-15,77993.0,70934.0,0.0,0.0,Sabziwala,148927.0,0.91,0.0,0.0,0.91
3,ST Push,5e8b11e4e3ca782085182ce1,later,0,custom_filters,21600.0,2020-11-12,3344.0,2457.0,0.0,0.0,TVwala,5801.0,0.73,0.0,0.0,0.73
6,TXN API Push,5fc17fda119bd90684d75d03,soon,SMV,User,43200.0,2020-11-27,2.0,1.0,0.0,0.0,Armwala,3.0,0.50,0.0,0.0,0.50
13,General Push,5fb11cd3bc2f79154e7ca549,later,0,custom_filters,129600.0,2020-11-16,14759.0,10643.0,0.0,0.0,Gharwala,25402.0,0.72,0.0,0.0,0.72
17,ST Push,5f8eda9c97223f714dce0ad8,later,0,allusers,129600.0,2020-11-26,1092.0,537.0,0.0,0.0,Phonewala,1629.0,0.49,0.0,0.0,0.49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166936,ST Push,5f51dc9e4371644051943e19,later,0,allusers,129600.0,2020-11-25,292916.0,115285.0,0.0,0.0,Phonewala,408201.0,0.39,0.0,0.0,0.39
166937,ST Push,5f59016fb7f37c13ecaeb066,later,DMV,allusers,129600.0,2020-11-19,49829.0,21796.0,0.0,0.0,Phonewala,71625.0,0.44,0.0,0.0,0.44
166941,General Push,5fa3ab20a6e83c4fc286faa3,later,0,custom_filters,86400.0,2020-11-05,4612.0,3667.0,0.0,0.0,Life_max,8279.0,0.80,0.0,0.0,0.80
166942,ST Push,5f59016fb7f37c13ecaeb066,later,DMV,allusers,129600.0,2020-11-10,33001.0,15124.0,0.0,0.0,Phonewala,48125.0,0.46,0.0,0.0,0.46


### We bifurcate the dataframe into 2 parts, one of the campaigns using PushAmp+ and other of those nthat don't use PushAmp+

In [32]:
#Not using PA+
df1 = df[df['paplus_sent'] == 0]

# Dropping the columns that are either all 0 or inconsequential to analysis
df1 = df1.drop(['paplus_sent', 'imp', 'PSent/TS', 'PImp/PSent', 'cid'], axis = 1)
df1.head()

,ctype,delivery_type,sherpa_flag,target_segment,ttl,date,sent,impressions,app_name,chk_flag,NPImp/NPSent,TImp/TS
2,TXN API Push,soon,SMV,User,43200.0,2020-11-15,77993.0,70934.0,Sabziwala,148927.0,0.91,0.91
3,ST Push,later,0,custom_filters,21600.0,2020-11-12,3344.0,2457.0,TVwala,5801.0,0.73,0.73
6,TXN API Push,soon,SMV,User,43200.0,2020-11-27,2.0,1.0,Armwala,3.0,0.50,0.50
13,General Push,later,0,custom_filters,129600.0,2020-11-16,14759.0,10643.0,Gharwala,25402.0,0.72,0.72
17,ST Push,later,0,allusers,129600.0,2020-11-26,1092.0,537.0,Phonewala,1629.0,0.49,0.49


In [33]:
#Using PA+
df2 = df[df['paplus_sent'] != 0]

# cid is inconsequential
df2 = df2.drop(['cid'], axis = 1)
df2.head()

,ctype,delivery_type,sherpa_flag,target_segment,ttl,date,sent,impressions,paplus_sent,imp,app_name,chk_flag,NPImp/NPSent,PSent/TS,PImp/PSent,TImp/TS
87,General Push,soon,0,custom_filters,21600.0,2020-11-14,343279.0,208296.0,30626.0,27067.0,Songswala,551575.0,0.58,0.09,0.88,0.61
391,General Push,later,0,custom_filters,10800.0,2020-11-19,6264511.0,3277868.0,883599.0,726008.0,Songswala,9542379.0,0.47,0.14,0.82,0.52
481,General Push,later,0,custom_filters,54000.0,2020-11-05,357847.0,248129.0,81346.0,64661.0,Malay_prod,605976.0,0.66,0.23,0.79,0.69
485,General Push,later,0,custom_filters,14400.0,2020-11-27,878610.0,416784.0,97170.0,77107.0,Songswala,1295394.0,0.43,0.11,0.79,0.47
685,General Push,later,0,custom_filters,54000.0,2020-11-03,4896.0,3398.0,955.0,745.0,Malay_prod,8294.0,0.67,0.20,0.78,0.69


### Presenting the analysis

In [34]:
temp = df1.pivot_table(values = ['TImp/TS'], index = ['app_name'], aggfunc = 'mean')
temp.sort_values('TImp/TS')

,TImp/TS
app_name,
bikebuddy,0.000000
uae_cluster_uat,0.212889
Phonewala,0.436212
ABC_Prod,0.440583
Songswala,0.457097
Scooterportal,0.522710
Style_IND,0.594072
paintIND,0.615165
TVwala,0.635968


In [35]:
# apps not using PushAmp+
temp.describe().T

,count,mean,std,min,25%,50%,75%,max
TImp/TS,38.0,0.713008,0.205117,0.0,0.645131,0.73902,0.859092,1.0


In [36]:
# Analysing performance across apps
temp = df2.pivot_table(values = ['TImp/TS', 'NPImp/NPSent', 'PImp/PSent', 'PSent/TS'], index = ['app_name'], aggfunc = 'mean')
temp.sort_values('TImp/TS')


,NPImp/NPSent,PImp/PSent,PSent/TS,TImp/TS
app_name,,,,
Phonewala,0.321667,0.125000,0.000000,0.321667
TVwala,0.424086,0.760000,0.133978,0.469247
Songswala,0.434583,0.791245,0.108845,0.472901
Malay_prod,0.636955,0.761992,0.213625,0.662898
astro_hindi,0.872769,0.604615,0.136615,0.842462


In [37]:
# app using PushAmp+
temp.describe().T

,count,mean,std,min,25%,50%,75%,max
NPImp/NPSent,5.0,0.538012,0.219280,0.321667,0.424086,0.434583,0.636955,0.872769
PImp/PSent,5.0,0.608570,0.280042,0.125000,0.604615,0.760000,0.761992,0.791245
PSent/TS,5.0,0.118613,0.077058,0.000000,0.108845,0.133978,0.136615,0.213625
TImp/TS,5.0,0.553835,0.201747,0.321667,0.469247,0.472901,0.662898,0.842462


#### We observe that very few apps are using PushAmp+, this indicates huge opportunity to cross-sell PushAmp+ to them, as they are already onborad with us and will be easier to approach and close the deal quickly

#### Conclusion
        We observed that the apps which are not using PushAmp+ have mean of 71% delivery rate. And the apps, using PushAmp+ have delivery rate of about 53%. This means, that only the apps whose campaigns are struggling are willing to pay for PushAmp+ service.
        Now among the apps which choose to use PushAmp+ are still using it conservatively and sending around 11% of their notifications using PushAmp+. Although, the percentages is less but the PushAmp+ messages themselves observed 60% delivery rate. This means, had all messages been sent through PushAmp+, the apps could have seen 60% delivery rate, that is, the delivery uplift of 7%.
        Nonetheless we observed 2% increase in overall delivery rate.

#### Analyzing ctype with sherpa_flag

In [38]:
temp = df1.pivot_table(values = ['TImp/TS'], index = ['ctype', 'sherpa_flag'], aggfunc = 'mean')
temp = temp.round(2)
temp

TImp/TS
ctype        sherpa_flag         
General Push 0               0.61
             DMV             0.43
             SMV             0.46
ST Push      0               0.70
             DMV             0.46
             SMV             0.58
TXN API Push 0               0.00
             SMV             0.75

In [39]:
temp = df2.pivot_table(values = ['TImp/TS', 'NPImp/NPSent', 'PImp/PSent', 'PSent/TS'], index = ['ctype', 'sherpa_flag'], aggfunc = 'mean')
temp = temp.round(2)
temp
#temp.sort_values('TImp/TS')

NPImp/NPSent  PImp/PSent  PSent/TS  TImp/TS
ctype        sherpa_flag                                             
General Push 0                    0.50        0.77      0.15     0.54
             DMV                  0.34        0.24      0.04     0.34
             SMV                  0.65        0.54      0.09     0.66
ST Push      0                    0.69        0.76      0.23     0.71
             DMV                  0.49        0.00      0.00     0.49
             SMV                  0.98        0.90      0.19     0.97
TXN API Push SMV                  0.56        0.83      0.09     0.58

#### Conclusion
        In case of Dynamic Multivariate (DMV) mode, we observed that very less percentage of PushAmp+ messages were sent, 4% and 0% in General Push and ST Push respectively. Even in those that were sent the delivery rate of PushAmp+ was less than non-PushAmp+. This indicates, that the PushAmp+ is not compatible with DMV or we can say SherpaAI.
        Excluding DMV, in other modes, the mean delivery rate of PushAmp+ is 76%, which is better than non-PushAmp+ which is 67%

#### Analyzing ctype with Delivery type

In [40]:
# A Table
temp = df1.pivot_table(values = ['TImp/TS'], index = ['ctype', 'delivery_type'], aggfunc = 'mean')

temp = temp.round(2)
temp

TImp/TS
ctype        delivery_type         
General Push later             0.59
             soon              0.63
ST Push      later             0.63
             soon              0.61
TXN API Push soon              0.74

In [41]:
# B Table
temp = df2.pivot_table(values = ['TImp/TS', 'NPImp/NPSent', 'PImp/PSent', 'PSent/TS'], index = ['ctype', 'delivery_type'], aggfunc = 'mean')
temp = temp.round(2)
temp

NPImp/NPSent  PImp/PSent  PSent/TS  TImp/TS
ctype        delivery_type                                             
General Push later                  0.51        0.75      0.15     0.55
             soon                   0.42        0.80      0.11     0.47
ST Push      later                  0.75        0.78      0.22     0.76
TXN API Push later                  0.56        0.83      0.09     0.58

#### Conclusion
        Here we again observed, that PushAmp+ is being used in scenario where delievry rate is low, by comparing TImp/Ts from Table A with NPImp/NPSent from table B. 
        From Table B, we observed around 80% in PushAmp+ which clearly provide Delivery uplift albiet the percentage of PushAmp+ messages are low.

#### Analyzing Time of live campaign with Impressions

In [42]:
temp = df1.pivot_table(values = ['TImp/TS'], index = ['ttl'], aggfunc = 'mean')
temp = temp.round(2)
temp

,TImp/TS
ttl,
0.0,0.57
1800.0,0.42
3600.0,0.44
7200.0,0.53
10800.0,0.42
14400.0,0.66
18000.0,0.77
21600.0,0.77
28800.0,0.58


In [43]:
temp = df2.pivot_table(values = ['TImp/TS', 'NPImp/NPSent', 'PImp/PSent', 'PSent/TS'], index = ['ttl'], aggfunc = 'mean')
temp = temp.round(2)
temp

,NPImp/NPSent,PImp/PSent,PSent/TS,TImp/TS
ttl,,,,
60.0,0.13,0.37,0.11,0.16
2400.0,0.32,0.57,0.12,0.34
3600.0,0.43,0.66,0.16,0.46
5400.0,0.38,0.63,0.11,0.41
6300.0,0.42,0.71,0.11,0.45
7200.0,0.44,0.74,0.12,0.47
9000.0,0.38,0.79,0.10,0.42
10800.0,0.46,0.78,0.11,0.49
14400.0,0.46,0.82,0.12,0.50


#### Conclusion
        One may feel that with longer campaign time, delivery rate may increase but we observed no such relation.

## Major Obervation:
    1. We can cross sell PushAmp+ to our existing customers who are not using PushAmp+.
    2. We observed that PushAmp+ is being used only by customers who are facing low delivery rate.
    3. Even the customers who are using PushAmp+, are using it conservatively. This could indicate that the users are price sensitive and are inclined to use PushAmp+ when absolutely necessary. 
    4. #2 and #3 seem to indicate that the prices of PushAmp+ are not user friendly and it seems PushAmp+ is being used in last resort kind of situation. This hints on a need to reformulate pricing strategy.
    5. PushAmp+ and SherpaAI don't seem to be compatible.

    